# Lista 7 z Wielowymiarowej Analizy Statystycznej

## Setup

In [1]:
import numpy as np
import pandas as pd

corr_full = np.array([
    [1,     0.72, -0.44, 0.60],
    [0.72,  1,    -0.13, 0.68],
    [-0.44, -0.13, 1,    -0.29],
    [0.60,  0.68, -0.29, 1]
])

In [2]:
std_devs = np.array([11.228, 14.404, 10.926, 3.095])
std_devs

array([11.228, 14.404, 10.926,  3.095])

## Zadanie 1a: macierz korelacji cząstkowych warunkowanych po jednej zmiennej

### Sposób macierzowy
Macierze wyznaczamy korzystając ze wzoru podanego na liście 6:
$$\rho_{ij|1...i-1,i+1,...,j-1,j+1,...n} = \frac{-C_{ij}}{\sqrt{C_{ii}C_{jj}}},$$
gdzie $C_{ij}$ jest dopełnieniem algebraicznym elementu $ij$ w macierzy korelacji zmiennych $X_1, ... X_n$. W podanym wzorze warunkujemy po wszystkich pozostałych zmiennych, więc żeby dostosować to do zadania, jeśli chcemy warunkować tylko po jednej zmiennej $X_k$ musimy jako $C$ podać podmacierz macierzy korelacji składającą się tylko z wierzy i kolumn zmiennych $X_i,X_j,X_k$.

In [3]:
def get_minor_matrix(A: pd.DataFrame, i, j):
    "Zwraca macierz powstałą przez wykreślenie wiersza i i kolumny j z macierzy A."
    B = A.to_numpy()
    result =  np.delete(np.delete(B, [j], axis = 1), [i], axis = 0)
    return result

def get_cofactor(A: pd.DataFrame, i, j):
    """Zwraca dopełnienie algebraiczne elementu NAZWANEGO i, j macierzy A (pd.DataFrame)"""
    # znajdujemy numery indeksów i,j w uciętej macierzy
    i_ = np.where(A.index == i)[0][0]
    j_ = np.where(A.index == j)[0][0]
    A_ = get_minor_matrix(A, i_, j_)
    return np.linalg.det(A_)*(-1)**(i_+j_)

def get_partial_ij_ks(corr: np.array, i: int, j: int, ks: list) -> float:
    """Zwraca wartość korelacji zmiennych i,j pod warunkiem zmiennych ks."""
    # dla przekątnej zwracamy 1
    if i == j:
        return 1.00
    # przesuwamy indeksy o 1 dla zgodności z zadaniem i tworzymy uciętą macierz
    named_corr = pd.DataFrame(corr)
    named_corr.columns = range(1,len(corr)+1) 
    named_corr.index = range(1,len(corr)+1)
    reduced_corr = named_corr.loc[[i,j] + ks,[i,j] + ks]
    
    # obliczamy dopełnienia algebraiczne
    C_ii = get_cofactor(reduced_corr, i, i)
    C_jj = get_cofactor(reduced_corr, j, j)
    C_ij = get_cofactor(reduced_corr, i, j)
    return - C_ij / (np.sqrt(C_ii*C_jj))


#### Zadanie 1a.i.

In [4]:
# wypełnienie macierzy z wynikami
partial_corrs = pd.DataFrame(np.zeros((3,3)))
partial_corrs.columns = [1,2,4]
partial_corrs.index = [1,2,4]
for i in partial_corrs:
    for j in partial_corrs.index:
        partial_corrs.loc[i,j] = get_partial_ij_ks(corr_full,i,j,[3])

partial_corrs.round(2)

,1,2,4
1,1.00,0.74,0.55
2,0.74,1.00,0.68
4,0.55,0.68,1.00


In [19]:
# Wersja na kolosa
def get_partial_ij_ks_2(corr_full: pd.DataFrame, i: int, j: int, ks: list) -> float:
    """Zwraca wartość korelacji zmiennych i,j pod warunkiem zmiennych ks."""
    # dla przekątnej zwracamy 1
    if i == j:
        return 1.00
    corr_reduced = corr_full.loc[[i,j] + ks,[i,j] + ks]
    i_ = np.where(corr_reduced.index == i)[0][0]
    j_ = np.where(corr_reduced.index == j)[0][0]
    C_ij = np.linalg.det(corr_reduced.drop(i, axis = 0).drop(j, axis = 1).to_numpy()) * (-1)**(i_+j_)
    C_ii = np.linalg.det(corr_reduced.drop(i, axis = 0).drop(i, axis = 1).to_numpy()) * (-1)**(i_+i_)
    C_jj = np.linalg.det(corr_reduced.drop(j, axis = 0).drop(j, axis = 1).to_numpy()) * (-1)**(j_+j_)
    return - C_ij / (np.sqrt(C_ii*C_jj))

pandas_corr = pd.DataFrame(corr_full).set_axis([1,2,3,4], axis = 0).set_axis([1,2,3,4], axis = 1)
partial_corrs = pd.DataFrame(np.zeros((3,3)))
partial_corrs.columns = [1,2,4]
partial_corrs.index = [1,2,4]
for i in partial_corrs:
    for j in partial_corrs.index:
        partial_corrs.loc[i,j] = get_partial_ij_ks_2(pandas_corr,i,j,[3])

partial_corrs.round(2)

,1,2,4
1,1.00,0.74,0.55
2,0.74,1.00,0.68
4,0.55,0.68,1.00


#### Zadanie 1a.ii.

In [5]:
# wypełnienie macierzy z wynikami
partial_corrs = pd.DataFrame(np.zeros((3,3)))
partial_corrs.columns = [1,2,3]
partial_corrs.index = [1,2,3]
for i in partial_corrs:
    for j in partial_corrs.index:
        partial_corrs.loc[i,j] = get_partial_ij_ks(corr_full,i,j,[4])

partial_corrs.round(2)

,1,2,3
1,1.00,0.53,-0.35
2,0.53,1.00,0.10
3,-0.35,0.10,1.00


## Zadanie 1b: współczynnik korelacji cząstkowej zbioru wyników warunkowany dwoma zmiennymi
Warunkujemy po zmiennych $X_3, X_4$ równocześnie - używamy funkcji z poprzedniego zadania.

In [6]:
# wypełnienie macierzy z wynikami
partial_corrs = pd.DataFrame(np.zeros((2,2)))
partial_corrs.columns = [1,2]
partial_corrs.index = [1,2]
for i in partial_corrs:
    for j in partial_corrs.index:
        partial_corrs.loc[i,j] = get_partial_ij_ks(corr_full,i,j,[3,4])

partial_corrs.round(2)

,1,2
1,1.00,0.61
2,0.61,1.00


**Komentarz:** Korelacja pomiędzy zmiennymi 1 i 2 wynosi około 0.61.

## Zadanie 1c: Współczynnik korelacji wielokrotnej
Współczynnik korelacji wielokrotnej wyraża współzależność pomiędzy jedną zmienną losową a wektorem losowym, tj. maksymalną możliwą do uzyskania korelację Pearsona pomiędzy zmienną $X_1$ a dowolną kombinacją liniową cech $X_2, ... X_n$:
$$R_{1|2...n} = \sup_{a \in R^{n-1}} corr(X_1, a^T (X_2,...X_n)).$$

Obliczamy ją ze wzoru macierzowego
$$R_{i|1...i-1,i+1,...n} = \sqrt{1 - \frac{\det C}{C_{ii}}},$$
gdzie $C$ to pełna macierz korelacji $corr(X_i,X_j)$ a $C_{ii}$ jest dopełnieniem algebraicznym jej elementu $ii$.

In [7]:
def get_multiple_correlation(corr: np.array, i: int, ks: list = None) -> float:
    """Zwraca współczynnik korelacji wielokrotnej pomiędzy zmienną i a zbiorem cech ks."""
    # przesuwamy indeksy o 1 dla zgodności z zadaniem i tworzymy uciętą macierz
    named_corr = pd.DataFrame(corr)
    named_corr.columns = range(1,len(corr)+1) 
    named_corr.index = range(1,len(corr)+1)
    # jeśli nie podano ks, to weź wszystko oprócz i
    if ks is None:
        ks = [k for k in named_corr.index.values if k != i]
    reduced_corr = named_corr.loc[[i] + ks,[i] + ks]
    det_C = np.linalg.det(reduced_corr.to_numpy())
    C_ii = get_cofactor(reduced_corr, i, i)
    return np.sqrt(1 - det_C/C_ii)

In [8]:
get_multiple_correlation(corr_full, 1).round(2) # i.

0.8

In [9]:
get_multiple_correlation(corr_full, 2).round(2) # ii.

0.81

In [10]:
get_multiple_correlation(corr_full, 2, [3,4]).round(2) # iii.

0.68

## Zadanie 1d: Wektor regresji cechy 1 na 2,3,4
**UWAGA! W zadaniu mamy macierz KORELACJI, a korzystamy tu z macierzy KOWARIANCJI, więc trzeba najpierw je przekonwertować!**
Wektor regresji $X$ na $Y$ obliczamy ze wzoru
$$E(X|Y = y) = EX + \Sigma_{XY}\Sigma_{YY}^{-1}(y - EY),$$
gdzie $\Sigma$ to podmacierze blokowe macierzy korelacji wektora $(X,Y)^T$. Ponieważ nie znamy tutaj konkretnych wartości $Y$, tzn. wektora $(X_2, X_3, X_4)$ ani wartości oczekiwanych, wyliczamy po prostu macierz współczynników regresji, tj.
$$\Sigma_{XY}\Sigma_{YY}^{-1}.$$

Macierz korelacji zamieniamy na macierz kowariancji następująco:
$$corr(X,Y) = \frac{cov(X,Y)}{\sigma_X \sigma_Y} \implies cov(X,Y) = corr(X,Y)\sigma_X \sigma_Y,$$
tzn. musimy przemnożyć wiersz $i$ przez $\sigma_i$ a kolumnę $j$ przez $\sigma_j.$ Robimy to w sposób macierzowy.

In [4]:
diagonal_stds = np.diag(std_devs)
diagonal_stds

array([[11.228,  0.   ,  0.   ,  0.   ],
       [ 0.   , 14.404,  0.   ,  0.   ],
       [ 0.   ,  0.   , 10.926,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  3.095]])

In [5]:
cov_full = diagonal_stds @ corr_full @ diagonal_stds # mnożymy z lewej i prawej przez to samo
cov_full

array([[126.067984  , 116.44424064, -53.97793632,  20.850396  ],
       [116.44424064, 207.475216  , -20.45915352,  30.3146584 ],
       [-53.97793632, -20.45915352, 119.377476  ,  -9.8066313 ],
       [ 20.850396  ,  30.3146584 ,  -9.8066313 ,   9.579025  ]])

In [6]:
def get_regression_matrix(cov, xs, ys):
    """Zwraca macierz współczynników regresji podwektora xs na podwektor ys w oparciu o macierz KOWARIANCJI."""
    # przesuwamy indeksy o 1 dla zgodności z zadaniem i wydzielamy macierze blokowe
    named_cov = pd.DataFrame(cov)
    named_cov.columns = range(1,len(cov)+1) 
    named_cov.index = range(1,len(cov)+1)
    Sigma_XY = named_cov.loc[xs,ys].to_numpy()
    Sigma_YY = named_cov.loc[ys,ys].to_numpy()
    return Sigma_XY @ np.linalg.inv(Sigma_YY)

get_regression_matrix(cov_full, [1], [2,3,4]).round(2)

array([[ 0.49, -0.35,  0.29]])

## Zadanie 1e: Współczynniki korelacji kanonicznej (tego ma nie być)
Korelację kanoniczną definiujemy jako maksymalną korelację liniową pomiędzy kombinacjami liniowymi współrzędnych tych wektorów, tzn. podobnie do wielokrotnej ale teraz $X_1$ jest wektorem (zmiana nazwy na $X$, $X_2 \rightarrow Y$):
$$\sup_{a \in R^p, b \in R^q} corr(a^T X, b^T Y),$$
gdzie $X, Y$ są wektorami rozmiarów $p$ i $q$. Współczynniki $a$ i $b$ możemy znaleźć w następujący sposób:

1. Wyznaczamy macierz $S = \Sigma_{XX}^{-1}\Sigma_{XY}\Sigma_{YY}^{-1}\Sigma_{YX}.$
2. Obliczamy wartości własne $\rho^2_1, ..., \rho^2_p$ macierzy $S$. 

**Uwaga: Dokładnie $z = ord(\Sigma_{XY}) = min(p,q)$ z nich powinno być niezerowe. Ponadto są to te same wartości własne, co macierzy $S'$ obliczonej analogicznie z zamianą indeksów 1 i 2 miejscami.**

3. Pierwiastki z wartości własnych są nazywane współczynnikami korelacji kanonicznej.

In [12]:
def compute_canonical_correlation(corr: np.array, xs: list, ys: list):
    """Obliczawspółczynniki korelacji kanonicznej i pośrednie kroki między zestawami cech xs i ys."""
    # przesuwamy indeksy o 1 dla zgodności z zadaniem i wydzielamy macierze blokowe
    named_corr = pd.DataFrame(corr)
    named_corr.columns = range(1,len(corr)+1) 
    named_corr.index = range(1,len(corr)+1)
    Sigma_XX = named_corr.loc[xs,xs].to_numpy()
    Sigma_YY = named_corr.loc[ys,ys].to_numpy()
    Sigma_XY = named_corr.loc[xs,ys].to_numpy()
    Sigma_YX = named_corr.loc[ys,xs].to_numpy() 
    assert np.all(Sigma_XY == Sigma_YX.T)
    # Wyznaczamy macierze Sa i Sb
    Sa = np.linalg.inv(Sigma_XX) @ Sigma_XY @ np.linalg.inv(Sigma_YY) @ Sigma_YX
    Sb = np.linalg.inv(Sigma_YY) @ Sigma_YX @ np.linalg.inv(Sigma_XX) @ Sigma_XY
    # Znajdujemy wartości własne tych macierzy (auto posortowane)
    Sa_vals, _ = np.linalg.eig(Sa)
    Sb_vals, _ = np.linalg.eig(Sb)
    assert np.all(Sa_vals >= 0)
    assert np.all(Sb_vals >= 0)
    # Niezerowe wartosci wlasne
    Sa_nz_vals = Sa_vals[Sa_vals > 10**(-14)] # dokładność numeryczna
    Sb_nz_vals = Sb_vals[Sb_vals > 10**(-14)]
    assert np.all(np.abs(Sa_nz_vals - Sb_nz_vals) < 10**(-14))
    # Wspolczynniki korelacji kanonicznej
    a = np.sqrt(Sa_nz_vals)
    b = np.sqrt(Sb_nz_vals)
    print(f"a = {a.round(2)}")
    print(f"b = {b.round(2)}")

In [15]:
compute_canonical_correlation(corr_full, [1,2], [3,4]) # Czy one są takie same?

a = [0.48 0.7 ]
b = [0.48 0.7 ]
